In [10]:
import os


class Para:
    read_folder_path = 'data/zj_space_1d_rate_20d_17-21_raw'
    write_folder_path = 'data/zj_space_1d_rate_20d_17-21_pre'

    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    month_num = len(os.listdir(read_folder_path))
para = Para()
print(para.month_num)

1197


In [11]:
import time
import numpy as np
import pandas as pd


time_begin = time.time()

df = pd.DataFrame()

for i_csv in range(para.month_num):

    df_curr_month = pd.read_csv(para.read_folder_path +'/'+ str(i_csv) + '.csv', index_col='order_book_id')

    df_curr_month['date'] = pd.to_datetime(df_curr_month['date'])

    df = pd.concat([df, df_curr_month])

df

,board_type,sector_code,yield_rate,month,date,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
order_book_id,,,,,,,,,,,,,,,,,,,,,
600895.XSHG,MainBoard,Industrials,-0.013492,0,2017-01-03,0.019066,3.596055,0.105905,-0.192159,-0.214464,...,47.058824,55.555556,-0.531171,0.510848,0.590319,0.967413,0.222327,0.460197,89.709172,97.931034
600895.XSHG,MainBoard,Industrials,-0.016778,1,2017-01-04,0.018970,3.614248,0.105372,-0.174930,-0.206557,...,58.870968,55.555556,0.072760,0.496079,0.586948,0.964116,0.207044,0.464071,100.000000,106.082725
600895.XSHG,MainBoard,Industrials,-0.010064,2,2017-01-05,0.018960,3.616269,0.105313,-0.158640,-0.196973,...,55.263158,55.555556,0.249363,0.485891,0.588574,0.961362,0.179641,0.468016,97.387173,104.854369
600895.XSHG,MainBoard,Industrials,0.000563,3,2017-01-06,0.019098,3.589991,0.106084,-0.154439,-0.188466,...,33.333333,55.555556,-0.325514,0.478257,0.586710,0.949731,0.145590,0.473251,105.852417,110.769231
600895.XSHG,MainBoard,Industrials,-0.009516,4,2017-01-09,0.018981,3.612226,0.105431,-0.140614,-0.178896,...,54.430380,66.666667,0.407361,0.457787,0.588179,0.941847,0.101692,0.476494,109.768638,111.479592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600895.XSHG,MainBoard,Industrials,0.019661,1192,2021-11-29,0.073442,2.024455,0.081535,-0.367241,-0.411104,...,26.415094,44.444444,-1.791065,0.414774,0.580053,0.869180,0.144255,1.069519,59.281437,48.784722
600895.XSHG,MainBoard,Industrials,0.013459,1193,2021-11-30,0.072898,2.039552,0.080931,-0.353182,-0.399519,...,38.613861,44.444444,-1.009226,0.416210,0.574087,0.828973,0.148582,1.087710,55.490196,54.480287
600895.XSHG,MainBoard,Industrials,0.010067,1194,2021-12-01,0.072703,2.045042,0.080714,-0.334951,-0.386606,...,38.000000,55.555556,-0.676599,0.416538,0.571355,0.787593,0.149042,1.104002,59.879032,54.757630


In [12]:
# Replace infinite updated data with nan
df.replace(to_replace=[np.inf, -np.inf], value=np.nan, inplace=True)

# 剔除含有 na 的行
df_dropna = df.dropna(axis=0, how='any')

# 选择特征列和标签列
df_dropna_y = df_dropna[['board_type', 'sector_code', 'month', 'date']]
df_dropna_features = df_dropna.loc[:, para.feature_column_start_name:para.feature_column_end_name]
df_dropna_features = pd.concat([df_dropna['yield_rate'], df_dropna_features], axis=1)

# 中位数去极值
df_dropna_features = df_dropna_features.clip(
    lower=df_dropna_features.median(axis=0) - 5 * df_dropna_features.mad(axis=0),
    upper=df_dropna_features.median(axis=0) + 5 * df_dropna_features.mad(axis=0),
    axis=1)

# 标准化
df_dropna_features = (df_dropna_features - df_dropna_features.mean()) / df_dropna_features.std(ddof=0)

# 拼接标签列和特征列
df_dropna_standard = pd.concat([df_dropna_y, df_dropna_features], axis=1)

# 写入文件
df_dropna_standard.to_csv(para.write_folder_path + '/0.csv')

print()
print('Done!')


Done!
